# W pętli zdarzeń - Asyncio

### Marcin Markiewicz
### 12.IV.2018, Python Level UP

![Logo kursu Python Level Up](https://raw.githubusercontent.com/daftcode/python_levelup_2018/master/logo.png)

![Plan zajęć](https://raw.githubusercontent.com/daftcode/python_levelup_2018/master/plan_zajec.png)

# Programowanie asyncio - po co to komu?

## Pytanie: jeśli teraz modne jest programowanie asynchroniczne, to co robiliśmy do tej pory?

Trzeba sobie zdać sprawę z tego, że procesory są bardzo szybkie. Dane wewnątrz struktur procesora przetwarzają się o rząd wielkości szybciej niż w pamięci ram. Dostęp do świata zewnętrzenego, dyski (nawet ssd), sieć, to są kolejne rzędy wielkości wolniej. W związku z tym, w programach, które komunikują się z siecią, procesor głównie czeka (idle time). 

## Jak odzyskać stracony czas

Stracony czas (oczekiwanie na synchronizację danych) można odzyskać. Możemy użyć "klasycznych" rozwiązań, czyli wielowątkowości lub wieloprocesowosći. Jednak te technologie wiążą się ze stosunkowo dużym kosztem przełączania procesów (switching context). Problem ten wzrasta wraz z obciążeniem systemu. Przy dużej ilości procesów, zaczyna być kosztowny, a także może stanowić główny czas pracy programu. 

Rozwiązaniem problemu przełączania jest eliminacja przełączania procesów. Ideą programowania asynchronicznego jest cykliczne wywoływanie procedur w pętli zdarzeń. W jednym procesie, bez narzutu związanego z przełączaniem procesów.

Asynchroniczność jest wspierana bezpośrednio przez kernel linuxowy. AIO (Asynchronous I/O) jest to api pozawalające na operacje wyjścia/wejścia bez blokowania na czas oczekiwania na dane. Zamiast tego, proces może kontynuować pracę, i sprawdzić za jakiś czas czy są dostępne wyniki operacji. 

Link dla zainteresowanych:
- https://oxnz.github.io/2016/10/13/linux-aio/
- https://www.ibm.com/developerworks/linux/library/l-async/

# Jak to robi Python

## Asyncio

Moduł asyncio jest implementacją asynchronicznych operacji wyjścia/wejścia. Oryginalnie moduł ten był zaimplementowany poprzez wykorzystanie generatorów (`yield` i `yield from`). Obecnie (Python 3.5+) są dedykowane słowa kluczowe takie jak: `await` i `async`. 

## W pętli zdarzeń

EventLoop jest podstawowym konceptem asynchronicznego programowania. Jest to pętla zdarzeń. Czyli cyklicznie wywoływanych `korutyn`. Pomysł ten przypomina spotkanie tematyczne grupy ludzi, w którym może odzywać się tylko ta osoba, która ma `token`. Przkazanie tokenu następuje tylko w miejscach umiesczenia `await` w kodzie programu. Jest to zgoła inna koncepcja od programowania klasycznych wątków/procesów, gdzie przełączanie następuje w dowolnym miejscu kodu pythonowego, programista nie wie gdzie ono nastąpi.


## Przykład

Zróbmy AB test pobierania danych z serwera http. Użyjemy strony http://httpbin.org. Dla pełnego efektu potrzebujemy odpowiedzi serwera, która jest stosunkowo długa. http://httpbin.org/delay/1 generuje odpowiedź serwera która jest generowana przez około 1s 

## Klasyczny request

```python
import requests
import time

def make_sync_requests(max, url):
    print('Sync requests')
    start_time = time.time()

    response = []
    for i in range(0, max):
        requests.get(url)
        response.append(i)

    print(response)
    execution_time = time.time() - start_time
    print(f'execution time: {execution_time}s')
```


## Asynchroniczny request

```python
import asyncio
import time

from aiohttp import ClientSession

async def make_asyncio_request(url, index, session):
    async with session.get(url) as response:
        return index

async def make_asyncio_requests(max, url):
    print('Asyncio requests')
    start_time = time.time()
    async with ClientSession() as session:
        tasks = [make_asyncio_request(url, index, session) for index in range(0, max)]
        responses = await asyncio.gather(*tasks)
    print(responses)
    execution_time = time.time() - start_time
    print(f'execution time: {execution_time}s')
```

# Przykład użycia asyncpg

```python
import asyncio
import asyncpg
import datetime

async def main():
    # Establish a connection to an existing database named "test"
    # as a "postgres" user.
    conn = await asyncpg.connect('postgresql://postgres@localhost/test')
    # Execute a statement to create a new table.
    await conn.execute('''CREATE TABLE users(id serial PRIMARY KEY,name text,dob date)''')
    # Insert a record into the created table.
    await conn.execute('''INSERT INTO users(name, dob) VALUES($1, $2)''', 'Bob', datetime.date(1984, 3, 1))
    # Select a row from the table.
    row = await conn.fetchrow('SELECT * FROM users WHERE name = $1', 'Bob')
    # *row* now contains
    # asyncpg.Record(id=1, name='Bob', dob=datetime.date(1984, 3, 1))
    # Close the connection.
    await conn.close()
    
asyncio.get_event_loop().run_until_complete(main())
```


# Od czego zacząć przygodę z asyncio

## Jak uruchomić korutynę

```python
import asyncio

async def main():
    await asyncio.sleep(10)

asyncio.get_event_loop().run_until_complete(main())   
```

## Uruchomienie wielu korutyn i czekanie na ich rezultaty

```python
import asyncio

async def important_task(index):
    await asyncio.sleep(10)
    return index

async def run():
    tasks = [
        important_task(index)
        for index in range(0, 10)
    ]
    print(await asyncio.gather(*tasks))

asyncio.get_event_loop().run_until_complete(run())
```
**Results:**
```python
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
```


## Uruchomienie wielu korutyn i wykorzystanie ich rezultatów jak tylko to możliwe


```python
import asyncio

async def important_task(index):
    await asyncio.sleep(index)
    return index

async def run():
    tasks = [important_task(index) for index in range(10, 0, -1)]
    results = []
    for future in asyncio.as_completed(tasks):
        result = await future
        results.append(result)
    print(results)

asyncio.get_event_loop().run_until_complete(run())
```

**Results:**
```python
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
```



# Rekursywne tworzenie tasków

```python
import asyncio

async def important_task(index):
    await asyncio.sleep(1)
    print(index)

async def run():
    loop = asyncio.get_event_loop()
    for index in range(10):
        loop.create_task(important_task(index))

asyncio.get_event_loop().run_until_complete(run())
```

**Results:**
```python
0
1
2
3
4
5
6
7
8
9
```

## Nieskończona pętla z obsługą sygnałów systemowych

```python
loop = self.get_event_loop()

# setup services, make db connection etc...
async def bootstrap():
    pass

# terminate connection to db, redis etc...
async def shutdown():
    pass

# stop the loop, cancel asyncio task etc...
def terminate():
    loop.stop()

# Initial events
loop.create_task(bootstrap())

# Register signals for graceful termination
for _signal in (signal.SIGINT, signal.SIGTERM):
    loop.add_signal_handler(_signal, terminate())
    
# Main loop start
try:
    loop.run_forever()
finally:
    loop.run_until_complete(shutdown())
    loop.close()
```

## Mój task jest długi, i blokuje przełączanie pętli

Rozwiązaniem jest podział taska na mniejsze kawałki, które będziemy wyowłaywać przez await lub jako osobne taski.
Mozna także użyć sztuczki: `await asyncio.sleep(0)`

```python
async def long_task():
    for strategy in strategies:
        strategy.make_decision()
        await.sleep(0)       
```

# Co dalej?


## Przydatne linki
1. https://docs.python.org/3/library/asyncio.html
1. https://hackernoon.com/asyncio-for-the-working-python-developer-5c468e6e2e8e
1. https://medium.com/python-pandemonium/asyncio-coroutine-patterns-beyond-await-a6121486656f
1. https://medium.com/@yeraydiazdiaz/asyncio-coroutine-patterns-errors-and-cancellation-3bb422e961ff
1. https://magic.io/blog/asyncpg-1m-rows-from-postgres-to-python/
1. https://magic.io/blog/uvloop-blazing-fast-python-networking/
1. https://github.com/channelcat/sanic
1. http://lucumr.pocoo.org/2016/10/30/i-dont-understand-asyncio/

## Włącz tryb debug

Możesz ustawić zmienną środowiskową `PYTHONASYNCIODEBUG=1` lub dodać w kodzie wywołanie `loop.set_debug()`.

## Asyncio ma przyzwoity domyślny logger

Należy tylko pamiętać o ustawieniu poziomu debug.

Przykładowa prosta konfiguracja

```python
import logging
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
chandler = logging.StreamHandler()
chandler.setFormatter(formatter)
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logger.addHandler(chandler)
```